In [17]:
"""
@author: amirjaved
"""
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Lambda,Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Input
import cv2
import matplotlib.image as mpimg
#helper class to define input shape and generate training images given image paths & steering angles
from utils import INPUT_SHAPE, batch_generator
np.random.seed(0)
import matplotlib.pyplot as plt

#### configuring the gpu usage
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 2GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=8192)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)



In [2]:
def preprocess(args,image):
#    image = image[60:-25, :, :]
    ##### args["input_shape"] is a tupple (height,width,channels)
    image = cv2.resize(image, (INPUT_SHAPE[1], INPUT_SHAPE[0]), cv2.INTER_AREA)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
    return image


In [3]:
def load_data(params):
    """
    Load training data and split it into training and validation set
    """
    #reads CSV file into a single dataframe variable

    #read log data file and renaming columns
    data_df = pd.read_csv(os.path.join(params['data_dir'], 'data.csv'), sep=',')
    data_df = data_df.rename(columns=({'Unnamed: 0':'image_path','dist_0':'distance','ABS_RX':'steering_angle','ABS_Y':'speed'}))

    #we'll store the camera images as our input data
    X = data_df[['image_path']].values
    #and our steering commands as our output data
    y = data_df['steering_angle'].values

    #now we can split the data into a training (80), testing(20), and validation set
    #thanks scikit learn
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=params['test_size'], random_state=0)

    return X_train, X_test, y_train, y_test

In [4]:
def build_model(params):
    """
    NVIDIA model used
    Image normalization to avoid saturation and make gradients work better.
    Convolution: 5x5, filter: 24, strides: 2x2, activation: ELU
    Convolution: 5x5, filter: 36, strides: 2x2, activation: ELU
    Convolution: 5x5, filter: 48, strides: 2x2, activation: ELU
    Convolution: 3x3, filter: 64, strides: 1x1, activation: ELU
    Convolution: 3x3, filter: 64, strides: 1x1, activation: ELU
    Drop out (0.5)
    Fully connected: neurons: 100, activation: ELU
    Fully connected: neurons: 50, activation: ELU
    Fully connected: neurons: 10, activation: ELU
    Fully connected: neurons: 1 (output)

    # the convolution layers are meant to handle feature engineering
    the fully connected layer for predicting the steering angle.
    dropout avoids overfitting
    ELU(Exponential linear unit) function takes care of the Vanishing gradient problem. 
    """
    model = Sequential()
    model.add(Lambda(lambda x: x/127.5-1.0, input_shape=INPUT_SHAPE))
    model.add(Conv2D(24, 5, 5, activation='elu',  input_shape=(2, 2)))
    model.add(Conv2D(36, 5, 5, activation='elu', input_shape=(2, 2)))
    model.add(Conv2D(48, 5, 5, activation='elu', input_shape=(2, 2)))
    #model.add(Conv2D(64, 3, 3, activation='elu'))
    #model.add(Conv2D(64, 3, 3, activation='elu'))
    model.add(Dropout(params['keep_prob']))
    model.add(Flatten())
    model.add(Dense(100, activation='elu'))
    model.add(Dense(50, activation='elu'))
    model.add(Dense(10, activation='elu'))
    model.add(Dense(1))
    model.summary()

    return model

In [5]:
params = {
    'data_dir': 'data/',
    'test_size': 0.2,
    'keep_prob': 0.5,
    'nb_epoch':10,
    'samples_per_epoch':2000,
    'batch_size': 40,
    'save_best_only':'true',
    'learning_rate': 1.0e-4
}
#print parameters
print('-' * 30)
print('Parameters')
print('-' * 30)
for key, value in params.items():
    print('{:<20} := {}'.format(key, value))
print('-' * 30)

------------------------------
Parameters
------------------------------
data_dir             := data/
test_size            := 0.2
keep_prob            := 0.5
nb_epoch             := 10
samples_per_epoch    := 2000
batch_size           := 40
save_best_only       := true
learning_rate        := 0.0001
------------------------------


In [6]:
#load data
X_train, X_test, y_train, y_test = load_data(params)
#build nvdia model
model = build_model(params)


### loading pretrained nvdia model
relative_path = "/model-169.h5" 
abs_path = os.path.abspath(os.getcwd()+relative_path)
model.load_weights(abs_path)
model.compile(loss='mean_squared_error', optimizer=Adam(lr=params["learning_rate"]))


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda (Lambda)              (None, 480, 640, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 96, 128, 24)       1824      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 19, 25, 36)        21636     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 5, 48)          43248     
_________________________________________________________________
dropout (Dropout)            (None, 3, 5, 48)          0         
_________________________________________________________________
flatten (Flatten)            (None, 720)               0         
_________________________________________________________________
dense (Dense)                (None, 100)               7

In [28]:
images_arr = []
for i in range(len(X_test)):
    img_path = X_test[i][0]
    image_ = mpimg.imread(params['data_dir']+str(img_path)+".jpg") 
    #image_ = preprocess(params,image_)
    images_arr.append(image_)
    #pred = model.predict(np.expand_dims(image, axis=0), batch_size = 1)
    #print([y_test[i]],pred[0])
    #prediction.append(pred)
predictions = model.predict(np.expand_dims(images_arr, axis=0), batch_size = 1)
    #prediction = np.array(prediction)
#np.savetxt("predictions.txt",prediction, delimiter=",")
#score = model.evaluate(data[""], Y, verbose=0)

ValueError: Error when checking input: expected lambda_input to have 4 dimensions, but got array with shape (1, 64, 224, 224, 3)

In [8]:
def draw_image_with_label(img, label, prediction=None):
    theta = label * 0.69 #Steering range for the car is +- 40 degrees -> 0.69 radians
    line_length = 50
    line_thickness = 3
    label_line_color = (255, 0, 0)
    prediction_line_color = (0, 0, 255)
    pil_image = image.array_to_img(img, K.image_data_format(), scale=True)
    print('Actual Steering Angle = {0}'.format(label))
    draw_image = pil_image.copy()
    image_draw = ImageDraw.Draw(draw_image)
    first_point = (int(img.shape[1]/2),img.shape[0])
    second_point = (int((img.shape[1]/2) + (line_length * math.sin(theta))), int(img.shape[0] - (line_length * math.cos(theta))))
    image_draw.line([first_point, second_point], fill=label_line_color, width=line_thickness)
    
    if (prediction is not None):
        print('Predicted Steering Angle = {0}'.format(prediction))
        print('L1 Error: {0}'.format(abs(prediction-label)))
        theta = prediction * 0.69
        second_point = (int((img.shape[1]/2) + (line_length * math.sin(theta))), int(img.shape[0] - (line_length * math.cos(theta))))
        image_draw.line([first_point, second_point], fill=prediction_line_color, width=line_thickness)
    
    del image_draw
    plt.imshow(draw_image)
    plt.show()